In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import pickle as pickle 
import sys
import os
from sklearn.utils import resample

In [ ]:
data = pd.read_csv('train.csv')  


In [ ]:
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [ ]:
pip install haversine

In [ ]:
import haversine as hs

distance = []
for row in data.iterrows():
  row_record = row[1]
  pick_loc = (row_record.pick_lat, row_record.pick_lon)
  drop_loc = (row_record.drop_lat, row_record.drop_lon)
  distance.append(hs.haversine(pick_loc, drop_loc))
data['distance'] = pd.Series(distance)

In [ ]:
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,distance
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct,5.092777
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct,3.168062
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct,6.305404
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct,0.861947
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct,8.147793


In [ ]:
data = data.dropna(axis=0,how='any')
data = data.drop(['drop_time','pickup_time','tripid'],axis=1)
data.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,distance
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,correct,5.092777
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,correct,3.168062
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,correct,6.305404
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,correct,0.861947
5,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,correct,24.207073


In [ ]:
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(['incorrect','correct'])
data['label']=le.transform(data.label)
data.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,distance
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,0,5.092777
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,0,3.168062
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,0,6.305404
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,0,0.861947
5,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,0,24.207073


In [ ]:
data.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,distance
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,0,5.092777
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,0,3.168062
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,0,6.305404
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,0,0.861947
5,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,0,24.207073


In [ ]:
clf = RandomForestClassifier(n_estimators = 250, random_state = 42, max_depth=None)

In [ ]:
y=data[['label']].values
X=data.drop('label',axis=1).values

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)


In [ ]:
clf.fit(X_train , y_train)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
prediction = clf.predict(X_val)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_val = accuracy_score(y_val,prediction)
print('\naccuracy_score on train dataset : ', accuracy_val)


accuracy_score on train dataset :  0.9504950495049505


In [ ]:
clf.fit(X,y) #fit the whole data set

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
test_data = pd.read_csv('test.csv')

distance = []
for row in test_data.iterrows():
  row_record = row[1]
  pick_loc = (row_record.pick_lat, row_record.pick_lon)
  drop_loc = (row_record.drop_lat, row_record.drop_lon)
  distance.append(hs.haversine(pick_loc, drop_loc))
test_data['distance'] = pd.Series(distance)

test_data=test_data.dropna(axis=0,how='any')
test = test_data.drop(['drop_time','pickup_time','tripid'],axis=1)
test.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,distance
0,10.5,924,42,2.4486,148,6.83454,79.8750,6.77490,79.8840,289.27,6.705711
1,10.5,4249,20,0.0000,91,6.91168,79.8723,6.55091,79.9706,1912.70,41.558571
2,10.5,1552,255,2.6588,23,6.92145,79.8478,6.90539,79.8989,394.00,5.916686
3,10.5,462,16,0.0000,198,6.77433,79.9416,6.80401,79.9407,154.32,3.301766
4,10.5,814,392,12.3692,69,6.97968,79.9130,6.98875,79.8914,147.47,2.588545


In [ ]:
predict_test = clf.predict(test.iloc[:,:].values)

In [ ]:
predict_test.shape

(8576,)

In [ ]:
sub=pd.DataFrame(predict_test,columns=['prediction'])

In [ ]:
for i in range(len(sub)):
    if sub['prediction'][i]==0:
        sub['prediction'][i]=1
    else:
        sub['prediction'][i]=0

In [ ]:
sub.head()

,prediction
0,1
1,1
2,1
3,1
4,1


In [ ]:
sub['tripid']=test_data.tripid


In [ ]:
sub.head()

,prediction,tripid
0,1,213284604
1,1,213286352
2,1,213293973
3,1,213294622
4,1,213298687


In [ ]:
final_submition=sub[['tripid','prediction']]
final_submition.head()

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


In [ ]:
final_submition.to_csv('final_submition_v4_randomFRST.csv',index=False)